<a href="https://colab.research.google.com/github/aworklab/everydayPython/blob/main/ex_20221205.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘의 목표
* Customer Analysis
* Iterators

In [1]:
import pandas as pd
import numpy as np
import plotnine as gg
from plotnine import ggplot, aes
import matplotlib.pyplot as plt
import seaborn as sns 
from datetime import datetime,date, time, timezone, timedelta 
import os
import re

from collections import defaultdict 
from functools import reduce 
import prophet as pr

# Customer Analysis

In [2]:
os.listdir()

['.config',
 'match_result_test.csv',
 '1_order_payments_dataset.csv',
 'bz_data_analyis.csv',
 'KPMG_VI_New_raw_data_update_final.csv',
 'mock_sales_data.csv',
 'ecommerce-data.csv',
 'sales_vs_discount.csv',
 '1_orders_dataset.csv',
 'groceries.csv',
 'mall_customers.csv',
 'scanner_data.csv',
 'online_retail.xlsx',
 'sales.csv',
 'test.csv',
 'online_retail_II.xlsx',
 'sales_data_with_stores.csv',
 'KPMG_VI_New_raw_data_update_final.xlsx',
 '1_customers_dataset.csv',
 'sample-sales-data.csv',
 'sample_data']

In [14]:
df = pd.read_csv('bz_data_analyis.csv')
df['order_date'] = pd.to_datetime(df['order_date'], infer_datetime_format=True)
df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.


,customer_id,product,quantity_ordered,price_each,purchase_address,month,year,day,city,code,sales,order_date,hour,minute,count
0,295665,Macbook Pro Laptop,1,1700.00,"136 Church St, New York City, NY 10001",12,2019,30,New York City (NY),NY,1700.00,2019-12-30 00:01:00,0,1,1
1,295666,LG Washing Machine,3,600.00,"562 2nd St, New York City, NY 10001",12,2019,29,New York City (NY),NY,600.00,2019-12-29 07:03:00,7,3,1
2,295667,USB-C Charging Cable,3,11.95,"277 Main St, New York City, NY 10001",12,2019,12,New York City (NY),NY,11.95,2019-12-12 18:21:00,18,21,1
3,295668,27in FHD Monitor,2,149.99,"410 6th St, San Francisco, CA 94016",12,2019,22,San Francisco (CA),CA,149.99,2019-12-22 15:13:00,15,13,1
4,295669,USB-C Charging Cable,1,11.95,"43 Hill St, Atlanta, GA 30301",12,2019,18,Atlanta (GA),GA,11.95,2019-12-18 12:38:00,12,38,1


In [15]:
df['quantity_ordered'] = ['1' if re.findall(r'C', str(a)) else a for a in df['quantity_ordered']]
df['quantity_ordered'] = df['quantity_ordered'].astype(int)

In [20]:
df['date'] = df['order_date'].to_numpy().astype('datetime64[D]')
df['month'] = df['order_date'].to_numpy().astype('datetime64[M]')
df['rev'] = df['quantity_ordered'] * df['price_each']

In [28]:
cnt = df.groupby(['customer_id', 'month'])['rev'].agg({'count','sum'}).reset_index().rename(columns = {'customer_id':'id','count':'freq', 'sum':'rev'})
cnt.head()

,id,month,freq,rev
0,141234,2019-01-01,1,700.00
1,141235,2019-01-01,1,14.95
2,141236,2019-01-01,1,23.98
3,141237,2019-01-01,1,149.99
4,141238,2019-01-01,1,11.99


In [37]:
info_f = pd.pivot_table(cnt, index = 'id', columns = 'month', values = 'freq').reset_index()
info_f = pd.melt(info_f, id_vars = 'id', var_name = 'month', value_name = 'freq').sort_values(['id','month'], ascending = [1,0])
info_f = info_f.groupby(['id']).head(3)
info_f = info_f.groupby(['id'])['freq'].sum().reset_index()
info_f.head()

,id,freq
0,141234,0.0
1,141235,0.0
2,141236,0.0
3,141237,0.0
4,141238,0.0


In [38]:
info_m = pd.pivot_table(cnt, index = 'id', columns = 'month', values = 'rev').reset_index()
info_m = pd.melt(info_m, id_vars = 'id', var_name = 'month', value_name = 'money').sort_values(['id','month'], ascending = [1,0])
info_m = info_m.groupby(['id']).head(3)
info_m = info_m.groupby(['id'])['money'].sum().reset_index()
info_m.head()

,id,money
0,141234,0.0
1,141235,0.0
2,141236,0.0
3,141237,0.0
4,141238,0.0


In [40]:
max_date = cnt['month'].max()
max_date

Timestamp('2020-01-01 00:00:00')

In [43]:
info_r = cnt.groupby(['id']).agg({'month': lambda x : (max_date - x.max())}).reset_index().rename(columns = {'month':'recency'})
info_r['recency'] = info_r['recency'].dt.days
info_r.head()

0    365
1    365
2    365
3    365
4    365
Name: recency, dtype: int64

In [45]:
info = pd.merge(pd.merge(info_r, info_f, 'left', 'id'), info_m, 'left', 'id')
info = info.assign(r = pd.qcut(info['recency'], 5, list('12345')),
                   f = pd.qcut(info['freq'].rank(method = 'first'), 5, list('54321')),
                   m = pd.qcut(info['money'].rank(method = 'first'), 5, list('54321'))
                   )
info.head()

,id,recency,freq,money,r,f,m
0,141234,365,0.0,0.0,5,5,5
1,141235,365,0.0,0.0,5,5,5
2,141236,365,0.0,0.0,5,5,5
3,141237,365,0.0,0.0,5,5,5
4,141238,365,0.0,0.0,5,5,5


# Iterators

In [46]:
users = [{'mail': 'gregorythomas@gmail.com', 'name': 'Brett Holland', 'sex': 'M', 'age': 73},
 {'mail': 'hintoncynthia@hotmail.com', 'name': 'Madison Martinez', 'sex': 'F', 'age': 29},
 {'mail': 'wwagner@gmail.com', 'name': 'Michael Jenkins', 'sex': 'M', 'age': 51},
 {'mail': 'daniel79@gmail.com', 'name': 'Karen Rodriguez', 'sex': 'F', 'age': 32},
 {'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F', 'age': 42}]
users

[{'mail': 'gregorythomas@gmail.com',
  'name': 'Brett Holland',
  'sex': 'M',
  'age': 73},
 {'mail': 'hintoncynthia@hotmail.com',
  'name': 'Madison Martinez',
  'sex': 'F',
  'age': 29},
 {'mail': 'wwagner@gmail.com',
  'name': 'Michael Jenkins',
  'sex': 'M',
  'age': 51},
 {'mail': 'daniel79@gmail.com',
  'name': 'Karen Rodriguez',
  'sex': 'F',
  'age': 32},
 {'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F', 'age': 42}]

In [47]:
{k : eval(f'reduce(lambda acc, cur : acc + [cur["{k}"]], users, [])') for k in list(users[0].keys())}

{'mail': ['gregorythomas@gmail.com',
  'hintoncynthia@hotmail.com',
  'wwagner@gmail.com',
  'daniel79@gmail.com',
  'ujackson@gmail.com'],
 'name': ['Brett Holland',
  'Madison Martinez',
  'Michael Jenkins',
  'Karen Rodriguez',
  'Amber Rhodes'],
 'sex': ['M', 'F', 'M', 'F', 'F'],
 'age': [73, 29, 51, 32, 42]}